In [9]:
import pandas as pd
import requests
import json
import datetime as dt
import os

# Generation Data

In [11]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'

os.listdir(json_file_path)

['loanda_data_curated_0.json',
 'loanda_data_curated_200000.json',
 'loanda_data_curated_400000.json',
 'loanda_data_curated_600000.json',
 'loanda_data_curated_800000.json']

In [12]:
json_file_path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/generation_data/'
df_generation = pd.DataFrame()
for data_chunks in os.listdir(json_file_path):
    path = json_file_path + data_chunks
    with open(path, "r") as json_file:
    # Load the JSON data into a Python object.
        json_data = json.load(json_file)
        pd_json_data = pd.DataFrame.from_dict(json_data)
        df_generation = pd.concat([df_generation,pd_json_data])

In [13]:
df_generation

,E3TimeStamp,PotenciaAtivaTotal,EnergiaTotal_DataLogger,EnergiaDiaria_DataLogger,RadiacaoGlobal,TempAr,TempModulo,VelocidadeVento,EnergiaTotal_LoteB,EnergiaDiaria_LoteB,...,EnergiaReativa_LoteC,EnergiaTotal_LoteD,EnergiaDiaria_LoteD,EnergiaReativa_LoteD,EnergiaTotal_LoteE,EnergiaDiaria_LoteE,EnergiaReativa_LoteE,EnergiaTotal_LoteF,EnergiaDiaria_LoteF,EnergiaReativa_LoteF
0,2021-08-17 07:42:20.090000000,288,413620,51,103.0,21.0,-10.0,1.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2021-08-17 07:43:20.043000000,297,413630,56,107.0,21.0,-10.0,2.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2021-08-17 07:44:20.037000000,305,413630,61,111.0,21.0,-10.0,4.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2021-08-17 07:45:20.127000000,316,413640,66,115.0,21.0,-10.0,3.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2021-08-17 07:46:20.027000000,326,413640,71,118.0,21.0,-10.0,5.0,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949868,2023-08-09 22:16:10.357000000,0,3376782,5634,-3.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949869,2023-08-09 22:17:10.407000000,0,3376782,5634,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949870,2023-08-09 22:18:10.380000000,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0
949871,2023-08-09 22:19:10.373000000,0,0,0,-4.0,21.0,19.0,3.0,4325685.6,5570.4,...,0.0,3384084.0,5543.3,0.0,3345551.4,5488.1,0.0,3376782.4,5634.4,0.0


# Weather Data

In [14]:
path = 'C:/Users/EMC/Documents/doutorado/doctorade/data/loanda/weather_data/loanda_weather_curated.json'
with open(path, "r") as json_file:
# Load the JSON data into a Python object.
    json_data = json.load(json_file)
    df_weather = pd.DataFrame.from_dict(json_data)

In [15]:
df_weather

,time_in_iso8601,temperature_2m_in_°C,relativehumidity_2m_in_%,precipitation_in_mm,weathercode_in_wmo code_x,cloudcover_in_%,cloudcover_low_in_%,cloudcover_mid_in_%,cloudcover_high_in_%,windspeed_10m_in_km/h,...,sunset_in_iso8601,precipitation_sum_in_mm,rain_sum_in_mm,precipitation_hours_in_h,windspeed_10m_max_in_km/h,windgusts_10m_max_in_km/h,winddirection_10m_dominant_in_°,shortwave_radiation_sum_in_MJ/m²,et0_fao_evapotranspiration_in_mm,time_of_sunlight_in_iso8601
0,2021-07-16 00:00:00,20.4,68.0,0.3,51.0,73.0,0.0,100.0,42.0,9.4,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
1,2021-07-16 01:00:00,19.1,79.0,0.8,53.0,70.0,0.0,100.0,35.0,9.0,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
2,2021-07-16 02:00:00,18.0,89.0,1.1,55.0,76.0,2.0,100.0,46.0,10.3,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
3,2021-07-16 03:00:00,17.4,94.0,1.4,61.0,80.0,8.0,100.0,44.0,10.7,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
4,2021-07-16 04:00:00,17.2,95.0,1.1,55.0,74.0,10.0,99.0,17.0,10.0,...,2021-07-16 18:04:00,20.4,20.4,24.0,10.7,38.5,172.0,4.14,0.89,10.866666666666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19624,2023-10-11 16:00:00,34.5,42.0,0.0,0.0,15.0,0.0,14.0,22.0,12.3,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19625,2023-10-11 17:00:00,34.0,42.0,0.0,0.0,16.0,0.0,20.0,12.0,11.3,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19626,2023-10-11 18:00:00,32.5,48.0,0.0,1.0,27.0,0.0,23.0,43.0,7.5,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667
19627,2023-10-11 19:00:00,30.9,51.0,0.0,1.0,40.0,0.0,28.0,76.0,8.7,...,2023-10-11 18:34:00,nan,nan,0.0,17.4,34.6,nan,nan,nan,12.516666666666667


# Concat weather e generation data

In [43]:
df_generation['E3TimeStamp'] = pd.to_datetime(df_generation['E3TimeStamp'], format='mixed')
df_generation['time_join_generation'] = df_generation['E3TimeStamp'].dt.date.astype(str) +' '+ df_generation['E3TimeStamp'].dt.hour.astype(str)
df_weather['time_in_iso8601'] = pd.to_datetime(df_weather['time_in_iso8601'], format='ISO8601')
df_weather['time_join_generation'] = df_weather['time_in_iso8601'].dt.date.astype(str) +' '+ df_weather['time_in_iso8601'].dt.hour.astype(str)

In [52]:
df_weather.columns

Index(['time_in_iso8601', 'temperature_2m_in_°C', 'relativehumidity_2m_in_%',
       'precipitation_in_mm', 'weathercode_in_wmo code_x', 'cloudcover_in_%',
       'cloudcover_low_in_%', 'cloudcover_mid_in_%', 'cloudcover_high_in_%',
       'windspeed_10m_in_km/h', 'is_day_in_', 'shortwave_radiation_in_W/m²',
       'direct_radiation_in_W/m²', 'diffuse_radiation_in_W/m²',
       'direct_normal_irradiance_in_W/m²', 'terrestrial_radiation_in_W/m²',
       'shortwave_radiation_instant_in_W/m²',
       'direct_radiation_instant_in_W/m²', 'diffuse_radiation_instant_in_W/m²',
       'direct_normal_irradiance_instant_in_W/m²',
       'terrestrial_radiation_instant_in_W/m²', 'time_day_for_join',
       'weathercode_in_wmo code_y', 'temperature_2m_max_in_°C',
       'temperature_2m_min_in_°C', 'temperature_2m_mean_in_°C',
       'apparent_temperature_max_in_°C', 'apparent_temperature_min_in_°C',
       'apparent_temperature_mean_in_°C', 'sunrise_in_iso8601',
       'sunset_in_iso8601', 'precipit